In [47]:
import requests
import json
import time
import pandas as pd

API_KEY = 'fnhdgteef55nyjm4cxpj3828'

# teams_url = f"https://api.sportradar.us/nfl/official/trial/v7/en/seasons/2021/REG/01/depth_charts.json?api_key={API_KEY}"
# team_data = requests.get(teams_url)
# team_info = team_data.json()['teams']
# team_ids = list(map(lambda x: {'id': x['id'], 'name': x['name']}, team_info))

# with open('team_ids.json', 'w') as f:
#     json.dump(team_ids, f)

with open('team_ids.json') as f:
    teams = json.load(f)
    
all_team_penalties = []
# for year in range(2010, 2022):
#     for team in teams:
#         try:
#             url = f"https://api.sportradar.us/nfl/official/trial/v7/en/seasons/{year}/REG/teams/{team['id']}/statistics.json?api_key={API_KEY}"
#             data = requests.get(url)
#             penalties = data.json()['record']['penalties']
#             penalties['name'] = team['name']
#             penalties['year'] = year
#             all_team_penalties.append(penalties)
#             time.sleep(1)
#         except Exception as error:
#             print(url)
#             print(data)
#             print(str(error))
#             time.sleep(1)
    
# df = pd.DataFrame(all_team_penalties)

# df.to_csv('penalties.csv', index=False)



In [71]:
import numpy as np
from plotnine import *
pd.set_option('display.max_rows', 500)

TEAMS = ['Raiders', 'Cowboys', '49ers', 'Vikings', 'Browns', 'Chargers', 'Cardinals',
 'Bills', 'Texans', 'Titans', 'Chiefs', 'Bears', 'Buccaneers', 'Panthers', 'Jets',
 'Jaguars', 'Patriots', 'Ravens', 'Eagles', 'Steelers', 'Dolphins', 'Lions',
 'Saints', 'Seahawks', 'Falcons', 'Football Team', 'Broncos', 'Colts', 'Packers',
 'Giants', 'Rams', 'Bengals']
YEARS = range(2010, 2022)

df = pd.read_csv('penalties_new.csv')

mean = df.groupby(['year', 'name']).mean().sort_values(by=['year', 'yards'], ascending=False)

mean = mean.reset_index()

total_rankings = []

for year in YEARS:
    year_rank = {'year': year}
    for team in TEAMS:
        val = mean[(mean['year'] == year) & (mean['name'] == team)]
        if year == 2021:
            print(val)
        rank = (list(val.index)[0] + 1) % 32
        year_rank[team] = rank or 32
    total_rankings.append(year_rank)
    
df = pd.DataFrame(total_rankings)
df.to_csv('cumulative_rankings_yards.csv', index=False)

     year           name  penalties   yards
0    2021        Raiders      124.0  1104.0
1    2021        Cowboys      127.0  1103.0
2    2021          49ers      102.0  1066.0
3    2021        Vikings      111.0  1043.0
4    2021         Browns      112.0  1035.0
5    2021       Chargers      116.0  1033.0
6    2021      Cardinals      114.0   990.0
7    2021          Bills      113.0   980.0
8    2021         Texans      114.0   978.0
9    2021         Titans      103.0   973.0
10   2021         Chiefs      111.0   925.0
11   2021          Bears      106.0   914.0
12   2021     Buccaneers       97.0   874.0
13   2021       Panthers      107.0   864.0
14   2021           Jets      103.0   860.0
15   2021        Jaguars      104.0   856.0
16   2021       Patriots       95.0   854.0
17   2021         Ravens      102.0   852.0
18   2021         Eagles      107.0   842.0
19   2021       Steelers      106.0   831.0
20   2021       Dolphins      105.0   825.0
21   2021          Lions      10

In [103]:
import pandas as pd

df = pd.read_csv('penalties_new.csv')

for idx, num in enumerate(df.groupby('year').mean().yards):
    new_row = {'year': YEARS[idx], 'name': 'Average', 'yards': round(num, 0)}
    df = df.append(new_row, ignore_index=True)
    

df.drop(columns=['penalties'], inplace=True)
df = df.set_index(['name', 'year'])['yards'].unstack()
df.to_csv('penalties_yards_with_average.csv')

year             2010    2011    2012    2013    2014    2015    2016    2017  \
name                                                                            
49ers           933.0  1014.0   960.0   845.0   937.0   823.0   831.0   988.0   
Average         814.0   867.0   851.0   845.0   888.0   940.0   928.0   924.0   
Bears           768.0   800.0   811.0   708.0   997.0   909.0   967.0   943.0   
Bengals         720.0   874.0   857.0  1000.0   812.0   917.0   726.0  1027.0   
Bills           633.0   743.0   871.0   952.0  1031.0  1249.0   960.0   872.0   
Broncos         944.0   842.0   805.0  1000.0  1045.0  1063.0   985.0   885.0   
Browns          675.0   847.0  1005.0   920.0   932.0  1083.0   736.0   924.0   
Buccaneers      837.0  1007.0   819.0  1136.0   939.0  1195.0   926.0   795.0   
Cardinals       870.0  1004.0   857.0   744.0   707.0   758.0   889.0   880.0   
Chargers        677.0   753.0   796.0   765.0   967.0   942.0   780.0   927.0   
Chiefs          771.0   916.

In [105]:
import pandas as pd

df = pd.read_csv('penalties_new.csv')

for idx, num in enumerate(df.groupby('year').mean().penalties):
    new_row = {'year': YEARS[idx], 'name': 'Average', 'penalties': round(num, 0)}
    df = df.append(new_row, ignore_index=True)
    

df.drop(columns=['yards'], inplace=True)
df = df.set_index(['year', 'name'])['penalties'].unstack()
df.to_csv('penalties_total_with_average.csv')

In [38]:
import pandas as pd

df = pd.read_csv('cumulative_rankings.csv')
top_5 = []
for row_num in range(32):
    top_5_tally = 0
    row = df.iloc[row_num]
    for year in YEARS:
        yearly_position = row[str(year)]
        if yearly_position <= 5:
            top_5_tally += 1
    top_5.append({'team': row['team'], 'num_seasons': top_5_tally})
    
df = pd.DataFrame(top_5)
df.to_csv('num_seasons_yards.csv', index=False)

